In [1]:
from splinter import Browser
from selenium.common.exceptions import WebDriverException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import time
import datetime as dt
import csv
import os

In [2]:
# Add the current date to the CSV file name
current_date_str = dt.datetime.now().strftime('%Y%m%d')
csv_file = f'scraped_dog_food_{current_date_str}.csv'

In [3]:
if not os.path.isfile(csv_file):
    with open(csv_file, 'a', newline='', encoding='utf-8') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['Product Name', 'Price', 'Bag Size', 'Description', 'Ingredients', 'Guaranteed Analysis', 'Scraping Date', 'Product Link'])

In [4]:
def close_modal(browser):
    modal_selector = 'div.Modal__Backdrop'
    
    # Check if the modal is present before proceeding
    if browser.is_element_present_by_css(modal_selector, wait_time=5):
        # Attempt to close the modal
        try:
            #print("ATTEND TO CLOSE MODAL")
            browser.find_by_css('span.Modal__Close').first.click()
            #print("CLOSED MODAL")
        except ElementClickInterceptedException:
            # If the click is intercepted, try using JavaScript to close the modal
            browser.execute_script("document.querySelector('.Modal__Backdrop').click()")

In [5]:
def close_cookie_banner(browser):
    cookie_banner_selector = 'div#cookieBanner'
    if browser.is_element_present_by_css(cookie_banner_selector, wait_time=10):
        #print("ATTEND TO CLOSE COOKIE")
        # Use JavaScript to set the display style to none and hide the cookie banner
        browser.execute_script(f"document.querySelector('{cookie_banner_selector}').style.display = 'none'")
        #print("CLOSED COOKIE")

In [6]:
def expand_accordion_button(browser, index):
    button_selector = f'button.accordionButton[aria-expanded="true"][aria-controls="undefined_content_{index}"]'
    if browser.is_element_present_by_css(button_selector, wait_time=5):
        try:
            browser.find_by_css(button_selector).first.click()
        except ElementClickInterceptedException:
            browser.execute_script(f"document.querySelector('{button_selector}').click()")

In [7]:
def extract_text_from_div(browser, index):
    script = f"return document.querySelector('div#undefined_content_{index}').innerHTML.trim();"
    
    try:
        div_html = browser.execute_script(script)

        # Parse HTML content with BeautifulSoup
        soup = BeautifulSoup(div_html, 'html.parser')

        # Remove specific CSS styles
        for style_tag in soup('style'):
            style_tag.decompose()

        # Extract text content from cleaned HTML
        div_text = soup.get_text(separator=' ', strip=True)

        if index == 0:
            print(f"Product Description: {div_text}")
        elif index == 1:
            print(f"Ingredients: {div_text}")
        elif index == 2:
            print(f"Guaranteed Analysis: {div_text}")

        return div_text
    except Exception as e:
        print(f"Error extracting text from <div_{index}>: {e}")
        return None

In [8]:
def scrape_product_page(browser):
    scraped_data = []
    browser.is_element_present_by_css('div.productContainer__productContainer', wait_time=10)

    # Get HTML of the product page and parse it with BeautifulSoup
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    product_name = soup.find('h1', class_='productHeader__title').text.strip()
    product_price = soup.find('div', class_='productDetails__price').text.strip()

    selected_variant_button = soup.find('button', class_='productDetails__selected')
    selected_variant = selected_variant_button.text.strip() if selected_variant_button else None

    print(f"Product Name: {product_name}")
    print(f"Product Price: {product_price}")
    print(f"Selected Variant: {selected_variant}")

    description = extract_text_from_div(browser, 0)
    ingredients = extract_text_from_div(browser, 1)
    guaranteed_analysis = extract_text_from_div(browser, 2)

    print("\n" + "=" * 70 + "\n")

    # Add the current date to the scraped_data dictionary
    current_date = dt.datetime.now().strftime('%Y-%m-%d')

    # Get the product link
    product_link = browser.url

    # Append data to the scraped_data list
    scraped_data.append({
        'Product Name': product_name,
        'Price': product_price,
        'Bag Size': selected_variant,
        'Description': description,
        'Ingredients': ingredients,
        'Guaranteed Analysis': guaranteed_analysis,
        'Scraping Date': current_date,  
        'Product Link': product_link 
    })

    # Write data to CSV
    with open(csv_file, 'a', newline='', encoding='utf-8') as file:
        csv_writer = csv.writer(file)

        for data in scraped_data:
            csv_writer.writerow([
                data.get('Product Name', ''),
                data.get('Price', ''),
                data.get('Bag Size', ''),
                data.get('Description', ''),
                data.get('Ingredients', ''),
                data.get('Guaranteed Analysis', ''),
                data.get('Scraping Date', ''),  
                data.get('Product Link', '')  
            ])

    # Clear the browser cookies to avoid interference with the next product page
    browser.cookies.delete()

    return scraped_data

In [9]:
def scrape_page(browser):
    # Wait for the product grids to be present
    browser.is_element_present_by_css('div.DynamicProductList__Table', wait_time=10)

    # Get HTML of the page and Parse the HTML with BeautifulSoup
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    # Find and click on each product card's link
    product_cards = soup.find_all('a', class_='ProductResultName', href=True)

    # Check if any product cards were found before proceeding
    if product_cards:
        for product_card in product_cards:
            # Extract the href attribute from the product card
            product_href = product_card['href']

            # Try to click the product card link using JavaScript
            try:
                print(f"https://www.petvalu.ca/{product_href}")
                browser.execute_script(f"document.querySelector('a.ProductResultName[href=\"{product_href}\"]').click()")

                # Call the function to scrape information from the individual product page
                scraped_data = scrape_product_page(browser)

            except Exception as e:
                print(f"Error clicking the product card link: {e}")

            # Go back to the previous page (the list of product cards)
            browser.back()

            # Close the modal and cookie banner on each subsequent page
            close_modal(browser)
            close_cookie_banner(browser)
    else:
        print("No product cards found on the page.")

    return soup

In [10]:
# Set up Splinter
browser = Browser('chrome', headless=False) 

# Starting URL
base_url = 'https://www.petvalu.ca/category/dog/dry-food/30-081318-003'
browser.visit(base_url)

In [11]:
num_pages_to_scrape = 10
page_counter = 1

while page_counter <= num_pages_to_scrape:
    print(f"Scraping information from page {page_counter}...\n")

    # Scrape information from the current product page
    scraped_data = scrape_page(browser)

    # Increment the page counter
    page_counter += 1

    # Navigate to the next page using JavaScript click
    next_page_link_selector = 'a.page_button.next.link'
    
    # Navigate to the next page using Splinter's built-in method
    if browser.is_element_present_by_css(next_page_link_selector, wait_time=5):
        browser.execute_script(f"document.querySelector('{next_page_link_selector}').click()")
        
        # Give the browser a moment to load the next page
        time.sleep(2)
    else:
        # Break out of the loop if the "Next" button is not found
        print("No 'Next' button found. Exiting...")
        break

    # Close the modal and cookie banner on each subsequent page
    close_modal(browser)
    close_cookie_banner(browser)

Scraping information from page 1...

https://www.petvalu.ca//product/performatrin-ultra-limited-ingredient-kangaroo-recipe-adult-dog-food/FCM07083CA
Product Name: performatrin Ultra Limited Ingredient Kangaroo Recipe Adult Dog Food
Product Price: $112.99
Selected Variant: 24 lb
Product Description: performatrin Ultra Limited Ingredient Kangaroo Recipe Adult Dog Food embraces holistic nutrition, focusing on functional, whole foods. This recipe offers exceptional, complete and balanced nutrition for pets with food sensitivities. Key Benefits: Single source animal protein and limited carbohydrate resources. Kangaroo is a highly nutritious lean red meat that provides essential protein and helps maintain strong muscles. Easily digestible Green Peas and Chickpeas. No Grains, Gluten, Chicken, Fish, Corn, Eggs, Dairy, Artificial Flavours, Colours or Preservatives.
Ingredients: Kangaroo, Peas, Chickpeas, Dried Peas, Lentils, Kangaroo Meal, Pea Protein, Canola Oil (preserve with Mixed Tocopherol

https://www.petvalu.ca//product/acana-classic-red-dog-food/FCM04344CA
Product Name: ACANA Classics Red Meat Recipe Dog Food
Product Price: $98.99
Selected Variant: 31.9 lb
Product Description: ACANA Classics Red Meat Recipe Dog Food is a meal thoughtfully crafted from start to finish with delicious, quality ingredients. The Classics dog food recipes is a great option for your pup as the first ingredient always has fresh or raw animal ingredients. With a balanced 50/50 split between rich meat ingredients and nutritious fruit, grains and vegetables, ACANA Classics Red Meat provides your dog with a filling and nutritious meal crafted from simple, wholesome ingredients. It also includes nutrient-rich ingredients that support three key health benefits, including immune function, healthy digestion, and healthy skin and coat. Feed your dog premium nutrition at a classic price with ACANA Classics Red Meat recipe. Key Benefits: Crafted with 50% premium animal ingredients* balanced with 50% whol

https://www.petvalu.ca//product/acana-singles-duck-pear-recipe-dog-food/FCM04873CA
Product Name: ACANA Singles Duck & Pear Recipe Dog Food
Product Price: $114.99
Selected Variant: 24 lb
Product Description: ACANA Singles Duck with Pear Recipe Dog Food every ingredient matters. Thats why ACANA Singles Limited Ingredient Diet Recipes are thoughtfully crafted with protein from a single animal source*, making them perfect for picky pups or dogs with food sensitivities! ACANA's Duck & Pear Recipe includes 60% protein-rich duck ingredients** balanced with 40% vegetables, fruit and nutrients*** like nutritious pears, and no grain ingredients****. It also includes no artificial preservatives, flavours or colours added in the kitchen. This recipe helps support 3 key health benefits for your dog, including healthy skin & coat, healthy digestion and ideal body condition. It helps your dog feel so good on the inside, youll see it on the outside! Key Benefits: Crafted with protein-rich ingredients 

https://www.petvalu.ca//product/purina-pro-plan-specialized-sensitive-skin-stomach-salmon-rice-formula/FCM01912CA
Product Name: Purina Pro Plan Specialized Sensitive Skin & Stomach Salmon & Rice Formula Adult Dog Food
Product Price: $96.99
Selected Variant: 30 lb
Product Description: Purina Pro Plan Specialized Sensitive Skin & Stomach Salmon & Rice Formula Adult Dog Food is nutrient-rich with salmon as the first ingredient. This formula contains prebiotic fiber and has a highly digestible formula and nourishes the beneficial bacteria found in your dog's intestine for their digestive health. Ingredients such as rice and oat meal provide easily digestible sources of carbohydrates for your dog. Meanwhile, fish oil ,rich in EPA one of the omega-3 fatty acids, helps support healthy joints and mobility, while omega-6 fatty acids and vitamin A nourish your dog's skin and keep his coat looking healthy. There is also no corn, wheat or soy. Key Benefits: Oat meal is easily digestible and gentle

Guaranteed Analysis: Crude protein 27% Fat content 16% Crude ash 7.5% Crude fibre 5% Moisture 12% Calcium 1.2% Phosphorus 0.9% Omega-6 fatty acids* 2.6% Omega-3 fatty acids* 1.1% DHA (docosahexaenoic acid) 0.4% EPA (eicosapentaenoic acid) 0.3%


https://www.petvalu.ca//product/go-solutions-sensitivities-limited-ingredient-grain-free-salmon-recipe-dog-food/FCM04167CA
Product Name: Go! Solutions Sensitivities Limited Ingredient Grain-Free Salmon Recipe Dog Food
Product Price: $99.99
Selected Variant: 22 lb
Product Description: Go! Solutions Sensitivities Limited Ingredient Grain Free Salmon Recipe Dog Food is a delicious food made with premium quality salmon as the single-source animal ingredient. Using as few additional ingredients as possible, this recipe is free from common food allergens, making it a great option for sensitive dogs. Made by pet nutrition experts in Ontario, Canada with only premium ingredients, you will love the quality and your dog will love the taste. Key Benefits:

...

Product Name: Blue Buffalo True Solutions Mobility Care Formula Adult Dog Food
Product Price: $98.99
Selected Variant: 22 lb
Product Description: BLUE True Solutions Mobility Care Formula Adult Dog Food supports your dog's mobility, naturally! Made with natural ingredients, enhanced with vitamins, minerals and other nutrients, plus glucosamine and chondroitin, clinically proven to support joint health. Key Benefits: With Lifesource Bits: This formula contains a precise blend of antioxidants, vitamins and minerals carefully selected by holistic veterinarians and animal nutritionists to support immune system health, life stage requirements, and a healthy oxidative balance NATURAL DOG FOOD: Formulated for the health and well-being of your adult dog and features the finest natural ingredients enhanced with vitamins and minerals It never contains chicken (or poultry) by-product meals, corn, wheat, soy, artificial flavours or preservatives
Ingredients: INGREDIENT STATEMENT:Deboned Chicken, C

https://www.petvalu.ca//product/blue-buffalo-wilderness-chicken-recipe-large-breed-adult-dog-food/FCM06867CA
Product Name: Blue Buffalo Wilderness Chicken Recipe Large Breed Adult Dog Food
Product Price: $83.99$93.99
Selected Variant: 24 lb
Product Description: Blue Buffalo Wilderness Chicken Recipe Large Breed Adult Dog Food is the protein-rich food your large breed dog was born to love. Created to satisfy ancestral cravings, it starts with high-quality deboned chicken and contains wholesome grains for truly balanced nutrition. Formulated specifically for large breed adult dogs, this food contains Glucosamine, Chondroitin, and EPA to support joint health and mobility. Key Benefits: PACKED WITH EVEN MORE REAL CHICKEN: A recipe made to satisfy your dog's innate cravings, this protein-rich food has evolved to contain even more high-quality deboned chicken INCLUDES WHOLESOME GRAINS: Complex carbohydrates from wholesome grains including oatmeal, barley, and brown rice ensure a truly balanc

https://www.petvalu.ca//product/wilder-harrier-sustainable-fish-formula-puppy-dog-food/FCM06717CA
Error clicking the product card link: Message: javascript error: Cannot read properties of null (reading 'click')
  (Session info: chrome=120.0.6099.234)
Stacktrace:
0   chromedriver                        0x000000010bcf9cc8 chromedriver + 4844744
1   chromedriver                        0x000000010bcf1243 chromedriver + 4809283
2   chromedriver                        0x000000010b8bf77d chromedriver + 411517
3   chromedriver                        0x000000010b8c5c90 chromedriver + 437392
4   chromedriver                        0x000000010b8c879e chromedriver + 448414
5   chromedriver                        0x000000010b959913 chromedriver + 1042707
6   chromedriver                        0x000000010b939cc2 chromedriver + 912578
7   chromedriver                        0x000000010b958d42 chromedriver + 1039682
8   chromedriver                        0x000000010b939a63 chromedriver + 911971
9  

https://www.petvalu.ca//product/royal-canin-small-weight-care-dog-food/FCM00274CA
Product Name: Royal Canin Small Weight Care Dog Food
Product Price: $92.99
Selected Variant: 13 lb
Product Description: Royal Canin Small Weight Care Dog Food is for adult and mature small breed dogs (9 to 22 lb) with a tendency to gain weight. Key Benefits: Ideal fiber blend to support the feeling of fullness. Adapted protein content to help maintain muscle mass for healthy weight loss. Enriched with L-carnitine. Hunger-satisfying nutrition with a rich taste, formulated to help keep him fit.
Ingredients: Corn, Chicken by-product Meal, Wheat Gluten, Corn Gluten Meal, Pea Fiber, Barley, Brewers Rice, Natural Flavors, Chicken Fat, Dried Plain Beet Pulp, Powdered Cellulose, Powdered Psyllium Seed Husk, Fish Oil, Vegetable Oil, Potassium Chloride, Calcium Carbonate, Salt, Sodium Tripolyphosphate, Fructooligosaccharides, Choline Chloride, Taurine, Vitamins [Dl-alpha Tocopherol Acetate (Source of Vitamin E), L-

https://www.petvalu.ca//product/royal-canin-breed-health-nutrition-pug-puppy-dog-food/FCM06887CA
Product Name: Royal Canin Breed Health Nutrition Pug Puppy Dog Food
Product Price: $36.99
Selected Variant: 2.5 lb
Product Description: Royal Canin Breed Health Nutrition Pug Puppy Dog Food is tailor-made nutrition created just for your purebred Pug puppy. This exclusive breed-specific diet is uniquely formulated for your Pug puppy, with the specific nutrients to help them thrive. The custom kibble is specially designed for your Pug's short, square muzzle, making it easy for them to pick up and chew. An exclusive mix of antioxidants including vitamin E supports their developing immune systems and keeps their bodies growing strong. Essential nutrients like EPA and DHA from fish oil help nourish healthy skin and coat. With high-quality proteins and prebiotics promote healthy digestion and optimal stool quality. Once your Pug is over 10 months old, transition them to Royal Canin Pug Adult dog 

In [ ]:
browser.quit()